## ETL:
#### 1- Extraccion archivo "movies_dataset"

In [ ]:
import pandas as pd

# Cargamos el archivo
movies_dataset_df = pd.read_csv("movies_dataset.csv")

# Realizamos una inspección inicial
print(movies_dataset_df.info())  # Información general de las columnas y tipos de datos

In [ ]:
movies_dataset_df.head()  # Primeras filas del DataFrame

#### 1- Extraccion archivo "credits"

In [ ]:
# Cargamos el archivo
credits_df = pd.read_csv("credits.csv")

# Realizamos una inspección inicial
print(credits_df.info())  # Información general de las columnas y tipos de datos

In [ ]:
credits_df.head()  # Primeras filas del DataFrame

#### 2- Transformacion "Movies_dataset"

Comenzamos eliminando las columnas innecesarias para cumplir las solicitudes del cliente

In [ ]:
# Eliminamos columnas innecesarias
movies_dataset_df = movies_dataset_df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

# Confirmamos la eliminación de las columnas
print(movies_dataset_df.columns)

Revisamos valores faltantes

In [ ]:
# Revisamos los valores faltantes
missing_values = movies_dataset_df.isnull().sum().sort_values(ascending=False) #recuento de valores nulos en orden descendente
print("Valores faltantes por columna:")
print(missing_values)

Creamos una columna retorno y rellenamos valores en columnas revenue y budget, segun requerimiento del cliente

In [ ]:
# Rellenamos valores nulos en las columnas 'revenue' y 'budget' con 0
movies_dataset_df['revenue'] = movies_dataset_df['revenue'].fillna(0)
movies_dataset_df['budget'] = movies_dataset_df['budget'].fillna(0)

# Confirmamos de que ya no hay valores nulos en 'revenue' y 'budget'
print("Valores nulos en 'revenue':", movies_dataset_df['revenue'].isnull().sum())
print("Valores nulos en 'budget':", movies_dataset_df['budget'].isnull().sum())

In [ ]:
# Convertimos 'budget' y 'revenue' a numérico, reemplazando valores no convertibles por 0
movies_dataset_df['budget'] = pd.to_numeric(movies_dataset_df['budget'], errors='coerce')
movies_dataset_df['revenue'] = pd.to_numeric(movies_dataset_df['revenue'], errors='coerce')

# Creamos la columna 'return' dividiendo 'revenue' entre 'budget'
movies_dataset_df['return'] = movies_dataset_df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

# Confirmamos la creación de la nueva columna
print(movies_dataset_df[['revenue', 'budget', 'return']].head())

Eliminamos nulos, transformamos fecha, en columna realese_date y creamos columna release_year

In [ ]:
movies_dataset_df["release_date"].isna().value_counts()

In [ ]:
# Eliminamos filas con valores nulos en la columna 'release_date'
movies_dataset_df = movies_dataset_df.dropna(subset=['release_date'])
# Confirmamos la eliminación
print("Valores nulos en 'release_date' después de la eliminación:", movies_dataset_df['release_date'].isnull().sum())

In [ ]:
# Convertimos 'release_date' al formato AAAA-mm-dd
movies_dataset_df['release_date'] = pd.to_datetime(movies_dataset_df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Creamos la columna 'release_year' extrayendo solo el año
movies_dataset_df['release_year'] = pd.to_datetime(movies_dataset_df['release_date'], errors='coerce').dt.year

# Confirmamos los cambios
print(movies_dataset_df[['release_date', 'release_year']].head())



Desanidamos valores anidadodos de columnas "belongs_to_collection", 'production_companies',"genres","production_countries","spoken_languages" utilizando libreria ast. Dejando solo los valores que consideramos importantes para cumplir con lo solicitado por el cliente

In [ ]:
movies_dataset_df["belongs_to_collection"].head()

In [ ]:
#Importamos la libreria ast (arboles de sintaxis abrstracta)
import ast 

# Convertimos a diccionario y extraemos el nombre 'name' de la colección si existe
movies_dataset_df['belongs_to_collection'] = movies_dataset_df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x).get('name') if pd.notnull(x) and isinstance(x, str) and x.startswith('{') else None)

In [ ]:
movies_dataset_df["belongs_to_collection"].head()

In [ ]:
movies_dataset_df["production_companies"].head()

In [ ]:
# Extraemos los nombres de las compañías de producción
movies_dataset_df['production_companies'] = movies_dataset_df['production_companies'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) and x.startswith('[') else None
)


In [ ]:
movies_dataset_df["production_companies"].head()

In [ ]:
movies_dataset_df["genres"].head()

In [ ]:
# Extramos los nombres de la columna géneros
movies_dataset_df['genres'] = movies_dataset_df['genres'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) else None)

In [ ]:
movies_dataset_df["genres"].head()

In [ ]:
movies_dataset_df["production_countries"].head()

In [ ]:
# Extraemos los nombres de las compañías de producción
movies_dataset_df['production_countries'] = movies_dataset_df['production_countries'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) and x.startswith('[') else None)

In [ ]:
movies_dataset_df["production_countries"].head(10)

In [ ]:
movies_dataset_df["spoken_languages"].head()

In [ ]:
movies_dataset_df['spoken_languages'] = movies_dataset_df['spoken_languages'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) and x.startswith('[') else None)

In [ ]:
movies_dataset_df["spoken_languages"].head()

Revisamos nuevamente el Data Frame

In [ ]:
movies_dataset_df

Seleccionamos los 5 idiomas mas relevantes para cumplir con los requerimientos y que el archivo quede mas liviano

In [ ]:
top_5_idiomas = movies_dataset_df["spoken_languages"].value_counts()
top_5_idiomas


In [ ]:
# Define los idiomas deseados
idiomas_deseados = ['English', 'Spanish', 'Français',"Italiano", 'Portuguese']

# Filtra el dataframe para mantener solo las filas que contienen los idiomas deseados en 'spoken_languages'
movies_dataset_df = movies_dataset_df[
    movies_dataset_df['spoken_languages'].apply(lambda idiomas: any(idioma in idiomas_deseados for idioma in idiomas) if idiomas else False)
]

# Revisa el resultado
movies_dataset_df


eliminamos 

Nuevamente revisamos el dataframe y vemos que sigue habendo columnas que no serian necesarias para cumplir con los requerimientos por lo que procedemos a eliminarlas

In [ ]:
#Revisamos las columnas esistentes en el dataframe
print(movies_dataset_df.columns)

In [ ]:
movies_dataset_df = movies_dataset_df.drop(columns=['overview', 'tagline', 'status', 'spoken_languages', 'original_language', 'popularity'])  

#### 2- Transformacion "credits"

Visualizamos los datos

In [ ]:
credits_df

In [ ]:
credits_df["cast"].head()

Extraemos los datos necesarios de las dos columnas para cumplir con los requerimientos

In [ ]:
#Importamos la libreria ast (arboles de sintaxis abstracta)
import ast 
# Creamos una funcion para extraer solo 'character' y 'name' de la columna 'cast'
def extraer_cast_info(cast_data):
    try:
        cast_list = ast.literal_eval(cast_data) # Intenta convertir de cadena a lista de diccionarios
        return [{'character': person['character'], 'name': person['name']} for person in cast_list] #De poder hacerlo el paso anterior, extrae solo 'character' y 'name'
    except (ValueError, TypeError):
        return None # De no poder convertir a diccionario por error de valor o tipo devuelve None

# Aplicamos la función al DF para crear una nueva columna 'cast_info'
credits_df['cast_info'] = credits_df['cast'].apply(extraer_cast_info)

In [ ]:
credits_df["cast_info"].head()

In [ ]:
# Creamos una unción para extraer sólo el nombre de los directores de la columna 'crew'
def extraer_director_info(crew_data):
    try:
        crew_list = ast.literal_eval(crew_data)# Intenta convertir de cadena a lista de diccionarios
        return [person['name'] for person in crew_list if person['job'] == 'Director'] #De poder hacerlo, filtra solo los directores y extrae sus nombres
    except (ValueError, TypeError): # De no poder convertir a diccionario por error de valor o tipo devuelve None
        return None

# Aplicamos la función al DF para crear una nueva columna 'director_info'
credits_df['director_info'] = credits_df['crew'].apply(extraer_director_info)

In [ ]:
credits_df['director_info'].head()

Limpiamos el data frame y dejamos solo las nuevas columnas con la informacion necesaria para las funciones

In [ ]:
credits_df.drop(columns=["cast","crew"],inplace=True)

In [ ]:
credits_df

Concatenamos los dos data frame

In [ ]:
Df_limpio = pd.concat([movies_dataset_df,credits_df],axis=1)

Guardamos el Df en un nuevo archivo "DF_final.csv"

In [ ]:
Df_limpio.to_csv("DF_final.csv",index=False)